In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [7]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns = ['has_covid']), df['has_covid'], test_size = 0.2)

In [10]:
x_train

,age,gender,fever,cough,city
51,11,Female,100.0,Strong,Kolkata
30,15,Male,101.0,Mild,Delhi
4,65,Female,101.0,Mild,Mumbai
92,82,Female,102.0,Strong,Kolkata
72,83,Female,101.0,Mild,Kolkata
...,...,...,...,...,...
15,70,Male,103.0,Strong,Kolkata
6,14,Male,101.0,Strong,Bangalore
98,5,Female,98.0,Strong,Mumbai
13,64,Male,102.0,Mild,Bangalore


In [12]:
# adding simple imputer to fever col
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [13]:
x_test_fever.shape

(20, 1)

In [14]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also the test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [15]:
# OneHotEncoding -> gender, city
ohe = OneHotEncoder(drop='first', sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [16]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

#also the test data
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

x_train_age.shape

(80, 1)

In [18]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough), axis = 1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

### Column Transformer

In [19]:
from sklearn.compose import ColumnTransformer

In [20]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [21]:
transformer.fit_transform(x_train).shape

(80, 7)

In [22]:
transformer.transform(x_test).shape

(20, 7)